### Horse or Human Classification using CNN

In [ ]:
#Load Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
import keras


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

Load Data

In [ ]:
train_data_path ='/content/drive/MyDrive/horse-or-human/Dataset/train'
validation_data_path ='/content/drive/MyDrive/horse-or-human/Dataset/validation'

### Data Preprocessing

In [ ]:
training_datagen=ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                    width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')

In [ ]:
training_data=training_datagen.flow_from_directory(train_data_path,
                                                   target_size=(150, 150), 
                                      batch_size=32,
                                      class_mode='binary')

Found 1037 images belonging to 2 classes.


In [ ]:
training_data.class_indices

{'horses': 0, 'humans': 1}

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
valid_data =  valid_datagen.flow_from_directory(validation_data_path, 
                                      target_size=(150, 150), 
                                      batch_size=32,
                                      class_mode='binary')

Found 256 images belonging to 2 classes.


### Building CNN Model

In [ ]:
cnn_model=keras.models.Sequential([
                                   keras.layers.Conv2D(filters=32,kernel_size=3,input_shape=[150,150,3]),#Add First CNN Layer
                                   keras.layers.MaxPooling2D(pool_size=(2,2)), #Add First MaxPooling Layer

                                   keras.layers.Conv2D(filters=64,kernel_size=3),#Add Second CNN Layer
                                   keras.layers.MaxPooling2D(pool_size=(2,2)), #Add Second MaxPooling Layer

                                   keras.layers.Conv2D(filters=128,kernel_size=3),#Add Third CNN Layer
                                   keras.layers.MaxPooling2D(pool_size=(2,2)), #Add Third MaxPooling Layer

                                   keras.layers.Conv2D(filters=256,kernel_size=3),#Add Fourth CNN Layer
                                   keras.layers.MaxPooling2D(pool_size=(2,2)), #Add Fourth MaxPooling Layer

                                   keras.layers.Dropout(0.5),# Droupout Layers
                                   keras.layers.Flatten(),

                                   keras.layers.Dense(units=128,activation='relu'), # First Hidden Layer
                                   keras.layers.Dropout(0.1), # Droupout Layers

                                   keras.layers.Dense(units=256,activation='relu'), # Second Hidden Layer
                                   keras.layers.Dropout(0.25), # Droupout Layers

                                   keras.layers.Dense(units=2,activation='softmax') #Output Layer
                                   


])

In [ ]:
# compile cnn model
cnn_model.compile(optimizer = Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_path='/content/drive/MyDrive/horse-or-human/horse_or_human_predictor.h5'
checkpoint=ModelCheckpoint(model_path,monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')
callbacks_list=[checkpoint]

## Train CNN Model

In [ ]:
history = cnn_model.fit(training_data, 
                          epochs=100, 
                          verbose=1, 
                          validation_data= valid_data,
                          callbacks=callbacks_list)

Epoch 1/100
33/33 [==============================] - 16s 383ms/step - loss: 0.8605 - accuracy: 0.5234 - val_loss: 0.6116 - val_accuracy: 0.6133

Epoch 00001: val_accuracy improved from -inf to 0.61328, saving model to /content/drive/MyDrive/horse-or-human/horse_or_human_predictor.h5
Epoch 2/100
33/33 [==============================] - 12s 374ms/step - loss: 0.6222 - accuracy: 0.6912 - val_loss: 1.4614 - val_accuracy: 0.5352

Epoch 00002: val_accuracy did not improve from 0.61328
Epoch 3/100
33/33 [==============================] - 12s 368ms/step - loss: 0.4926 - accuracy: 0.7584 - val_loss: 3.7436 - val_accuracy: 0.5156

Epoch 00003: val_accuracy did not improve from 0.61328
Epoch 4/100
33/33 [==============================] - 12s 366ms/step - loss: 0.4954 - accuracy: 0.7779 - val_loss: 1.8837 - val_accuracy: 0.5898

Epoch 00004: val_accuracy did not improve from 0.61328
Epoch 5/100
33/33 [==============================] - 12s 366ms/step - loss: 0.3920 - accuracy: 0.8112 - val_loss: 1.

### **Import Model**

In [ ]:
model_path='/content/drive/MyDrive/horse-or-human/horse_or_human_predictor.h5'

In [ ]:
model = keras.models.load_model(model_path)

In [ ]:
#horse image path
h1='/content/drive/MyDrive/horse-or-human/test_data/horse/1.jpg'
h2='/content/drive/MyDrive/horse-or-human/test_data/horse/2.jpg'
h3='/content/drive/MyDrive/horse-or-human/test_data/horse/3.jpeg'
h5='/content/drive/MyDrive/horse-or-human/test_data/horse/4.jpg'
h4='/content/drive/MyDrive/horse-or-human/test_data/horse/5.jpg'
#human image path
human1='/content/drive/MyDrive/horse-or-human/test_data/human/1.jpg'
human2='/content/drive/MyDrive/horse-or-human/test_data/human/2.jpg'
human3='/content/drive/MyDrive/horse-or-human/test_data/human/3.jpg'
human4='/content/drive/MyDrive/horse-or-human/test_data/human/4.jpg'
human5='/content/drive/MyDrive/horse-or-human/test_data/human/5.jpg'

In [ ]:
import numpy as np
from keras.preprocessing import image

In [ ]:
def pred_human_horse(model, horse_or_human):
  test_image = image.load_img(horse_or_human, target_size = (150, 150))
  test_image = image.img_to_array(test_image)/255
  test_image = np.expand_dims(test_image, axis = 0)

  result = model.predict(test_image).round(3)

  pred = np.argmax(result)
  print(result, "--->>>", pred)

  if pred == 0:
    print('Predicted>>> Horse')
  else:
    print('Predicted>>> Human')

### **Predict Output**

In [ ]:
for horse_or_human in [h1,h2,h3,h4,h5,human1,human2,human3,human4,human5]:
  pred_human_horse(model, horse_or_human)

[[0.997 0.003]] --->>> 0
Predicted>>> Horse
[[1. 0.]] --->>> 0
Predicted>>> Horse
[[0.533 0.467]] --->>> 0
Predicted>>> Horse
[[0. 1.]] --->>> 1
Predicted>>> Human
[[0.01 0.99]] --->>> 1
Predicted>>> Human
[[0.998 0.002]] --->>> 0
Predicted>>> Horse
[[0.796 0.204]] --->>> 0
Predicted>>> Horse
[[0. 1.]] --->>> 1
Predicted>>> Human
[[0.314 0.686]] --->>> 1
Predicted>>> Human
[[0.996 0.004]] --->>> 0
Predicted>>> Horse
